# Gromos Trajectory evaluation with Pygromos and Pandas

## Example file for the evaluation of GROMOS trajectory files in pygromos

1. Analysis of a GROMOS trc file (position trajectory)
    1. Import
    2. Common Functions
2. Analysis of a GROMOS tre file (energy trajectory)
    1. Import
    2. Common Functions

In [2]:
# general imports for manual data manipulations. Not needed if only provided functions are used
import numpy as np
import pandas as pd

In [3]:

#specific imports from pygromos for trc and tre file support
import pygromos.files.trajectory.trc as traj_trc
import pygromos.files.trajectory.tre as traj_tre



0	totene
1	totkin
2	totpot
3	totcov
4	totbond
5	totangle
6	totimproper
7	totdihedral
8	totcrossdihedral
9	totnonbonded
10	totlj
11	totcrf
12	totls
13	totlspair
14	totlsreal
15	totlsk
16	totlsa
17	totlsself
18	totlssurf
19	totpolself
20	totspecial
21	totsasa
22	totsasavol
23	totconstraint
24	totdisres
25	totdisfieldres
26	totdihres
27	totposres
28	totjval
29	totxray
30	totle
31	totorder
32	totsymm
33	eds_vr,entropy
34	totqm
35	totbsleus
36	totrdc
37	wip1
38	wip2
39	wip3
40	wip4
41	wip5
42	wip6


## 1) TRC

### 1.1) TRC import

In [4]:
# import the trajectory file into a Trc class
trc = traj_trc.Trc(input_value="example_files/Traj_files/test_CHE_vacuum_sd.trc")

/cluster/work/igc/wsalome/anaconda3/envs/pygromos/lib/python3.7/site-packages/pandas/core/generic.py:2621: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['POS_1', 'POS_2', 'POS_3', 'POS_4', 'POS_5', 'POS_6', 'POS_7', 'POS_8',
       'POS_9', 'POS_10', 'POS_11', 'POS_12', 'POS_13', 'POS_14', 'POS_15',
       'POS_16', 'POS_17', 'POS_18'],
      dtype='object')]

  encoding=encoding,


The Trc class offers the normal gromos block structure and additionally a pandas DataFrame called database where all the timesteps are stored.
For typical trc files the only classic block is the TITLE block, and all the other blocks are stored inside the database.

Additionally many common functions are offered to evaluate the given data. If a needed function is not provided, the normal pandas syntax can be used to create custom functions.

If you have a function that's generally useful, please contact the developers to possibly add it to the pygromos code to help other people :)

In [5]:
[x for x in dir(trc) if not x.startswith("_")]

['TITLE',
 'add_traj',
 'database',
 'get_atom_movement_length_mean',
 'get_atom_movement_length_series',
 'get_atom_movement_length_total',
 'get_atom_movement_series',
 'get_atom_pair_distance_mean',
 'get_atom_pair_distance_series',
 'get_cog_movement_total_series_for_atom_group',
 'get_cog_movement_vector_series_for_atom_group',
 'get_pdb',
 'path',
 'radial_distribution',
 'rmsd',
 'visualize',
 'write',
 'write_pdb']

### 1.2) Common trc functions

In [6]:
# Get the average movement length between two frames
trc.get_atom_movement_length_mean(atomI=1)

0.03803161677595912

In [7]:
# Or get the center of mass movement for a whole group of atoms. The atoms are provided as numbers in a list.
trc.get_cog_movement_total_series_for_atom_group(atoms=[1,2,5]).mean()

0.029597332762906308

In [8]:
# Get the average distance between two atoms over all time frames
trc.get_atom_pair_distance_mean(atomI=1, atomJ=2)

0.15259971796798974

#### RMSD

In [9]:
# Calculate the rmsd to the initial frame (0th frame).
# Alternatively a different trajectory can be provide as argument to the rmsd function.
# The accepted arguments are integer or single trajectory frame.
rmsd = trc.rmsd(0)

In [10]:
# Which returns the rmsd for every time frame to the initial frame.
# It can be seen how the rmsd slowly gets larger as the simulations get farther away from the initial setup.
rmsd

0      0.000000
1      0.029114
2      0.090643
3      0.136652
4      0.143205
         ...   
413    0.469253
414    0.473791
415    0.516979
416    0.510561
417    0.542082
Length: 418, dtype: float64

In [11]:
# The mean over all frames can be easily taken with the pandas function mean()
rmsd.mean()

0.45100360300648046

#### RDF

In [12]:
# This functionality is still under development

## 2) TRE

### 2.1) Tre import and structure

In [12]:
# import the trajectory file into a Tre class
from pygromos.files.trajectory.tre_field_libs.ene_fields import gromos_2015_tre_block_names_table

tre = traj_tre.Tre(input_value="example_files/Traj_files/test_CHE_H2O_bilayer.tre", _ene_ana_names=gromos_2015_tre_block_names_table)

/cluster/work/igc/wsalome/anaconda3/envs/pygromos/lib/python3.7/site-packages/pandas/core/generic.py:2621: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['totals', 'baths', 'bonded', 'nonbonded', 'special', 'eds', 'mass',
       'temperature', 'volume', 'pressure'],
      dtype='object')]

  encoding=encoding,


In [13]:
tre.database

,TIMESTEP_step,TIMESTEP_time,totals,baths,bonded,nonbonded,special,eds,mass,temperature,volume,pressure
0,0,0.0,"[-408806.2436, 393521.852, -802328.0956, 13840...","[[99739.83003, 29519.49412, 70220.33591], [293...","[[0.0, 49931.35526, 0.0, 88472.52195, 0.0]]","[[984602.6158, -1925334.589, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[296.2008302, 350.6569429, 278.0450574, 1.000...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[29.98293036, 94773.2037, 131128.599, 30.37611..."
1,1000,2.0,"[-411576.4224, 392980.7347, -804557.157, 13955...","[[99960.45845, 29481.77196, 70478.6865], [2930...","[[0.0, 50409.38587, 0.0, 89141.04865, 0.0]]","[[988489.6284, -1932597.22, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[296.8560381, 350.2088478, 279.0680247, 1.000...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[32.10481199, 92112.53213, 131040.7862, 26.183..."
2,2000,4.0,"[-409167.9172, 392370.1191, -801538.0363, 1389...","[[100096.9564, 29163.29693, 70933.65948], [292...","[[0.0, 50155.81989, 0.0, 88764.59053, 0.0]]","[[983688.8106, -1924147.257, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[297.2614008, 346.4257383, 280.8695397, 0.999...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[28.23515616, 96520.43393, 130756.5894, 30.172..."
3,3000,6.0,"[-408468.813, 392911.3266, -801380.1396, 13890...","[[100572.3051, 29097.77284, 71474.53225], [292...","[[0.0, 50205.91055, 0.0, 88694.38663, 0.0]]","[[984891.3076, -1925171.744, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[298.6730603, 345.647389, 283.0111843, 1.0000...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[28.79320593, 96009.21744, 130922.0286, 24.551..."
4,4000,8.0,"[-409322.7261, 391652.3813, -800975.1074, 1398...","[[100279.2719, 28656.33854, 71622.93335], [291...","[[0.0, 50608.00218, 0.0, 89216.65927, 0.0]]","[[980597.3587, -1921397.128, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[297.8028294, 340.4036676, 283.5987946, 0.999...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[24.7954137, 100472.9563, 130538.2987, 27.3050..."
5,5000,10.0,"[-409060.5321, 391503.6168, -800564.1489, 1401...","[[99621.54627, 28465.00254, 71156.54373], [291...","[[0.0, 50668.20147, 0.0, 89507.19113, 0.0]]","[[980518.6316, -1921258.173, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[295.8495588, 338.1308206, 281.7520741, 1.000...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[24.44895665, 100856.1354, 130501.386, 20.8121..."
6,6000,12.0,"[-407724.5301, 393376.3395, -801100.8696, 1399...","[[100618.1056, 29001.93804, 71616.16754], [292...","[[0.0, 50612.96859, 0.0, 89338.70979, 0.0]]","[[980975.7104, -1922028.258, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[298.8090756, 344.5089841, 283.5720046, 0.999...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[24.88400774, 100946.7969, 131119.5628, 21.646..."
7,7000,14.0,"[-406045.9433, 393472.141, -799518.0842, 14009...","[[101200.4734, 29288.55636, 71911.91706], [292...","[[0.0, 50652.38951, 0.0, 89438.21007, 0.0]]","[[979207.7071, -1918816.391, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[300.5385535, 347.9136733, 284.743057, 1.0000...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[22.9661595, 103318.3573, 131165.6623, 27.4988..."
8,8000,16.0,"[-408499.0398, 391678.3093, -800177.3491, 1397...","[[100402.1573, 28648.71805, 71753.43928], [291...","[[0.0, 50465.54724, 0.0, 89294.44028, 0.0]]","[[977305.7837, -1917243.12, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0.0],[1277287.736],"[[298.1677666, 340.3131452, 284.115547, 0.9999...","[2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...","[19.87564957, 106444.6562, 130544.6054, 14.226..."
9,9000,18.0,"[-406

In [15]:
[x for x in dir(tre) if not x.startswith("_")]

['ENEVERSION',
 'TITLE',
 'add_traj',
 'database',
 'get_Hvap',
 'get_density',
 'get_eds',
 'get_precalclam',
 'get_temperature',
 'get_totals',
 'get_totals_bonded',
 'get_totals_nonbonded',
 'get_totals_total',
 'path',
 'tre_block_name_table',
 'write']

Tre files contain all energy related data (like split up energy terms, temperature, pressure, .....). In PyGromos they generally share the same block structure as other files, but all the data inside the specific timesteps is stored efficiently inside a pandas DataFrame, here called tre.database . This database offers manipulation with all pandas functions. Alternatively many common functions are provided inside the Tre class. 

This class should in principle replace further usage of the gromos++ ene_ana function, since all these operation can be done efficiently on the pandas DataFrame. 

We are currently working on adding more common functions to the Tre class. If you find a useful function please contact the developers so the function can be added for general usage :)

### 2.2) Common Tre functions

In [16]:
# calculate the average density over all timesteps
tre.get_density().mean()

874.6182260652407

In [17]:
# calculate the mean temperature over all frames for all baths in the system. In this example two baths with slightly different temperatures.
tre.get_temperature().mean()

array([297.7702854 , 297.71343093])

Tables and lists inside the database are stored in numpy arrays. For example the two temperatures from the previous example are stored in a numpy array of size 2 since it has two temperature baths

Specific values inside a tre file can also be directly accessed with numpy and pandas syntax

In [18]:
tre.database.iloc[2]

TIMESTEP_step                                                 2000
TIMESTEP_time                                                  4.0
totals           [-409167.9172, 392370.1191, -801538.0363, 1389...
baths            [[100096.9564, 29163.29693, 70933.65948], [292...
bonded                 [[0.0, 50155.81989, 0.0, 88764.59053, 0.0]]
nonbonded                  [[983688.8106, -1924147.257, 0.0, 0.0]]
special          [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
eds                                                          [0.0]
mass                                                 [1277287.736]
temperature      [[297.2614008, 346.4257383, 280.8695397, 0.999...
volume           [2425.07286, 8.463596169, 0.0, 0.0, 0.0, 8.463...
pressure         [28.23515616, 96520.43393, 130756.5894, 30.172...
Name: 2, dtype: object

In [19]:
# select the first nonbonded energy value for the first force group over all time frames
tre.database["nonbonded"].apply(lambda x: x[0][0])

0    984602.6158
1    988489.6284
2    983688.8106
3    984891.3076
4    980597.3587
5    980518.6316
6    980975.7104
7    979207.7071
8    977305.7837
9    976722.0947
Name: nonbonded, dtype: float64

In [20]:
tre.get_totals()

(38,) [ -408806.2436    393521.852    -802328.0956    138403.8772
        0.         49931.35526        0.         88472.52195
        0.       -940731.9728    984602.6158  -1925334.589
        0.             0.             0.             0.
        0.             0.             0.             0.
        0.             0.             0.             0.
        0.             0.             0.             0.
        0.             0.             0.             0.
        0.             0.             0.             0.
        0.             0.     ]


ValueError: Shape of passed values is (10, 38), indices imply (10, 43)

,totene,totkin,totpot,totcov,totbond,totangle,totimproper,totdihedral,totcrossdihedral,totnonbonded,...,totjval,totxray,totle,totorder,totsymm,"eds_vr,entropy",totqm,totbsleus,totrdc,wip1
0,-408806.2436,393521.8520,-802328.0956,138403.8772,0.0,49931.35526,0.0,88472.52195,0.0,-940731.9728,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-411576.4224,392980.7347,-804557.1570,139550.4345,0.0,50409.38587,0.0,89141.04865,0.0,-944107.5915,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-409167.9172,392370.1191,-801538.0363,138920.4104,0.0,50155.81989,0.0,88764.59053,0.0,-940458.4467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-408468.8130,392911.3266,-801380.1396,138900.2972,0.0,50205.91055,0.0,88694.38663,0.0,-940280.4368,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-409322.7261,391652.3813,-800975.1074,139824.6615,0.0,50608.00218,0.0,89216.65927,0.0,-940799.7689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-409060.5321,391503.6168,-800564.1489,140175.3926,0.0,50668.20147,0.0,89507.19113,0.0,-940739.5415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-407724.5301,393376.3395,-801100.8696,139951.6784,0.0,50612.96859,0.0,89338.70979,0.0,-941052.5480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-406045.9433,393472.1410,-799518.0842,140090.5996,0.0,50652.38951,0.0,89438.21007,0.0,-939608.6838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-408499.0398,391678.3093,-800177.3491,139759.9875,0.0,50465.54724,0.0,89294.44028,0.0,-939937.3367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-406991.4101,392482.0724,-799473.4824,139902.9949,0.0,50693.74147,0.0,89209.25341,0.0,-939376.4773,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### $\lambda$-Sampling & TREs

In [ ]:
# import the trajectory file into a Tre class
tre = traj_tre.Tre(input_value="example_files/Traj_files/RAFE_TI_l0_5.tre")
tre.get_precalclam()

### EDS in TREs

In [ ]:
# import the trajectory file into a Tre class
tre = traj_tre.Tre(input_value="example_files/Traj_files/RAFE_eds.tre")
tre.get_eds()

## Concatenate  and Copy multiple Trajectories

Trajectories offer a wide range of additional file manipulations. Trajectory classes can be copied (deep) and added to each other to concatenate multiple small simulation pieces into one large trajectory. 

In [24]:
tre_copy = traj_tre.Tre(input_value=tre)

In [25]:
tre_copy.database.shape

(10, 12)

In [26]:
tre_combined = tre + tre_copy

In [27]:
tre_combined.database.shape

(19, 12)

In the new combined trajectory we have one long trajectory made from the two smaller ones. The length is one element shorter, since normally the last element of the first trajectory and the first element of the second trajectory is the same element. This can be controlled via the option "skip_new_0=True" in the add_traj() function which is the core of the "+" operator for trajectories. In the following line the default behavior can be seen as a smooth numbering in the TIMESTEPs.

In [28]:
tre_combined.database.TIMESTEP_time

0     0.00
1     0.04
2     0.08
3     0.12
4     0.16
5     0.20
6     0.24
7     0.28
8     0.32
9     0.36
10    0.40
11    0.44
12    0.48
13    0.52
14    0.56
15    0.60
16    0.64
17    0.68
18    0.72
Name: TIMESTEP_time, dtype: float64